# Problem Statement:

You want to collect text data using Twitter API.

https://developer.twitter.com/en

# AIML Batch2:

__Consumer API keys__

__API key:__

nucwiQk9GtsJfWFSGQjXFicnq

__API secret key:__

psbIvsTINgr6eEu89jMe2FwsOM28Cs3KaVLWUfBjADkAfFfhZV

__Access token:__

296102538-4rqAPJBZSl2dQW2Wd1qvciWfkNKFpv9GZ3IBp5t7

__Access token secret:__

FoCmyrWsWVeyxjksdRwqujVyzQGT9tZVjRGqJoav021MU

https://anaconda.org/conda-forge/tweepy

https://anaconda.org/anaconda/openpyxl

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import json

In [2]:
import tweepy
import openpyxl
from tweepy import OAuthHandler

In [3]:
# credentials

consumer_key = "nucwiQk9GtsJfWFSGQjXFicnq"
consumer_secret = "psbIvsTINgr6eEu89jMe2FwsOM28Cs3KaVLWUfBjADkAfFfhZV"
access_token = "296102538-4rqAPJBZSl2dQW2Wd1qvciWfkNKFpv9GZ3IBp5t7"
access_token_secret = "FoCmyrWsWVeyxjksdRwqujVyzQGT9tZVjRGqJoav021MU"

In [4]:
# call API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [5]:
# Lets provide the query which I need to search

query = "US election Joe Biden"

# Fetch tweets

Tweets = api.search(query, count=10, lang= 'en', exclude= 'retweets', tweet_mode= 'extended')

for tweet in Tweets:
    print(tweet)
    print("=================================================")

Status(_api=<tweepy.api.API object at 0x000001660C601D60>, _json={'created_at': 'Sat Jun 19 05:21:26 +0000 2021', 'id': 1406119911508828161, 'id_str': '1406119911508828161', 'full_text': '@mazemoore @Rachael1776 Fast forward to 2019.\n\nBiden: “China is going to eat our lunch? Come on, man.\n\nI mean, you know, they’re not bad folks, folks. But guess what? They’re not competition for us,”\n\nMmmm..what happened in the intervening years for you to change your mind Joe?\n\nhttps://t.co/uCbC7e6rKr', 'truncated': False, 'display_text_range': [24, 303], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'mazemoore', 'name': 'Maze', 'id': 1394680198705123329, 'id_str': '1394680198705123329', 'indices': [0, 10]}, {'screen_name': 'Rachael1776', 'name': 'Rachael Coleman 🇺🇸', 'id': 345019341, 'id_str': '345019341', 'indices': [11, 23]}], 'urls': [{'url': 'https://t.co/uCbC7e6rKr', 'expanded_url': 'https://www.nbcnews.com/politics/2020-election/biden-s-comments-downplay

# Getting the Tweets and some of its related Attributes

In [6]:
df = pd.DataFrame(columns=[
    'Tweets', 'User', 'User_statuses_count',
    'user_followers', 'User_location', 'User_verified',
    'fav_count', 'rt_count', 'tweet_date'
])
df

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date


In [7]:
def stream(data, file_name):
    i=0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [8]:
stream(data=["US election Joe Biden"], file_name='my_tweets')

In [9]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,RT @RYP__: TrumptyDumpty moves into phase ? of...,Willa A Bell,56576,208,,False,0,13,2021-06-19 07:11:08
1,RT @_su_gibi: Joe Biden really won an election...,Areeb Shariff,135919,1183,"Karachi, Pakistan",False,0,3,2021-06-19 06:33:07
2,@mazemoore @Rachael1776 Fast forward to 2019.\...,Victorian Dad,19759,859,Currently at Siege of Khartoum,False,0,0,2021-06-19 05:21:26
3,"RT @anniekarni: Tshibaka, who promoted false c...",Rosana,3651,14,,False,0,60,2021-06-19 04:50:35
4,"RT @anniekarni: Tshibaka, who promoted false c...",@amt0407,16478,341,,False,0,60,2021-06-19 03:38:55


In [10]:
df.shape

(1000, 9)

In [11]:
from textblob import TextBlob

In [12]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [13]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity>0:
        return 'Positive'
    elif analysis.sentiment.polarity==0:
        return 'Neutral'
    else:
        return 'Negative'

In [14]:
df['clean_tweet'] = df['Tweets'].apply(lambda x: clean_tweet(x))

In [15]:
df[['Tweets','clean_tweet']].head()

,Tweets,clean_tweet
0,RT @RYP__: TrumptyDumpty moves into phase ? of...,RT TrumptyDumpty moves into phase of his Seven...
1,RT @_su_gibi: Joe Biden really won an election...,RT su gibi Joe Biden really won an election in...
2,@mazemoore @Rachael1776 Fast forward to 2019.\...,Fast forward to 2019 Biden China is going to e...
3,"RT @anniekarni: Tshibaka, who promoted false c...",RT Tshibaka who promoted false claims of elect...
4,"RT @anniekarni: Tshibaka, who promoted false c...",RT Tshibaka who promoted false claims of elect...


In [16]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x: analyze_sentiment(x))

In [17]:
df[['clean_tweet','Sentiment']].head()

,clean_tweet,Sentiment
0,RT TrumptyDumpty moves into phase of his Seven...,Neutral
1,RT su gibi Joe Biden really won an election in...,Negative
2,Fast forward to 2019 Biden China is going to e...,Negative
3,RT Tshibaka who promoted false claims of elect...,Positive
4,RT Tshibaka who promoted false claims of elect...,Positive


In [18]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [19]:
df[['clean_tweet','Sentiment']].head()

,clean_tweet,Sentiment
0,RT TrumptyDumpty moves into phase of his Seven...,Neutral
1,RT su gibi Joe Biden really won an election in...,Negative
2,Fast forward to 2019 Biden China is going to e...,Negative
3,RT Tshibaka who promoted false claims of elect...,Positive
4,RT Tshibaka who promoted false claims of elect...,Positive


In [20]:
print(df.head())
print(df.shape)

                                              Tweets           User User_statuses_count user_followers                   User_location User_verified fav_count rt_count           tweet_date                                        clean_tweet Sentiment
0  RT @RYP__: TrumptyDumpty moves into phase ? of...   Willa A Bell               56576            208                                         False         0       13  2021-06-19 07:11:08  RT TrumptyDumpty moves into phase of his Seven...   Neutral
1  RT @_su_gibi: Joe Biden really won an election...  Areeb Shariff              135919           1183               Karachi, Pakistan         False         0        3  2021-06-19 06:33:07  RT su gibi Joe Biden really won an election in...  Negative
2  @mazemoore @Rachael1776 Fast forward to 2019.\...  Victorian Dad               19759            859  Currently at Siege of Khartoum         False         0        0  2021-06-19 05:21:26  Fast forward to 2019 Biden China is going to e...  Negative


In [21]:
n = 20
print('Original Tweet:\n'+df['Tweets'][n])
print()
print('Clean Tweet:\n'+df['clean_tweet'][n])
print()
print("Sentiment:\n"+df['Sentiment'][n])

Original Tweet:
RT @27khv: At his own presser, Joe Biden has just asked "how would we feel (Americans) if the rest of the world thought we interfered in de…

Clean Tweet:
RT At his own presser Joe Biden has just asked how would we feel Americans if the rest of the world thought we interfered in de

Sentiment:
Positive


In [22]:
n = 202
print('Original Tweet:\n'+df['Tweets'][n])
print()
print('Clean Tweet:\n'+df['clean_tweet'][n])
print()
print("Sentiment:\n"+df['Sentiment'][n])

Original Tweet:
RT @BombshellDAILY: TEXAS STATE BAR ASSOCIATION INVESTIGATING ABBOTT FOR ATTEMPTING TO REVERSE ELECTION 
Did bogus claims of fraud amount t…

Clean Tweet:
RT TEXAS STATE BAR ASSOCIATION INVESTIGATING ABBOTT FOR ATTEMPTING TO REVERSE ELECTION Did bogus claims of fraud amount t

Sentiment:
Neutral


In [23]:
n = 50
print('Original Tweet:\n'+df['Tweets'][n])
print()
print('Clean Tweet:\n'+df['clean_tweet'][n])
print()
print("Sentiment:\n"+df['Sentiment'][n])

Original Tweet:
RT @anniekarni: Tshibaka, who promoted false claims of election fraud last year and who has written articles supporting gay conversion ther…

Clean Tweet:
RT Tshibaka who promoted false claims of election fraud last year and who has written articles supporting gay conversion ther

Sentiment:
Positive


In [24]:
df[df.Sentiment=='Positive'].shape[0]

462

In [25]:
df[df.Sentiment=='Neutral'].shape[0]

308

In [26]:
df[df.Sentiment=='Negative'].shape[0]

230